In [ ]:
# Imports
import os

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import re
from unidecode import unidecode
from bs4 import BeautifulSoup as bs
from elasticsearch import Elasticsearch, helpers

from dataclasses import dataclass

import time

In [ ]:
# NOTE: Groot deel van warnings wordt veroorzaakt door certificaten die niet worden doorgegeven TODO

In [ ]:
# Elastic host
es = Elasticsearch('https://localhost:9200')

es = Elasticsearch(
    hosts=[
            "https://localhost:9200"
    ],
    http_auth=("elastic", "NES9DZ-QwhanXAQf9caV"),
#     use_ssl=True,
    verify_certs=False#,
#     ca_certs="./ca.crt",
)

# Set directory
os.chdir(r"C:\Users\Asher\Documents\School\_Scriptie\Data\Data_Extract")

# Print folder contents (should contain all .TEI folders)
os.listdir()

In [ ]:
# --------------------------------------------------------------------------------------
# Vereist list met folder van landen bv: ['ParlaMint-NL.TEI', "ParlaMint-BE.TEI"]
#
# ---Return list met dict [{Land1}, {Land 1}]---
#
# Voorbeeld voor land MET jaarfolders:
# item 1 --> 'land folder': [land xml] bv: 'ParlaMint-NL.TEI': ['ParlaMint-NL.xml']
# item rest --> 'ParlaMint-NL.TEI\\2014': ['ParlaMint-NL_2014-04-16-tweedekamer-2.xml', '...']
#
# Voorbeeld met land ZONDER jaarfolder:
# items (alle) --> {'ParlaMint-BG.TEI': ['ParlaMint-BG.xml', 'ParlaMint-BG_2014-10-27.xml', '...'}
#
# Ofwel len(dict) > 1 bij landen met jaarfolders, anders zit speciale file inbegrepen als eerste in dict
#
#  Path vinden met os.path.join(key, value[x])
# --------------------------------------------------------------------------------------

def get_xml_files(country_selection):
    os.chdir(r"C:\Users\Asher\Documents\School\_Scriptie\Data\Data_Extract")
    
    country_return_list = []

    # loop door alle folders die hierboven zijn geprint
    for country in os.listdir():

        # filter op specifiek land (IN BOX 2)
        if country in country_selection:
            paths_dict = {}

            # ga door alle inhoud van de landfolder heen
            for root, dirs, files in os.walk(country):
                file_data = []
                
                # loop door files van een folder
                for file in files:

                    #filter alleen de xml files
                    if ".xml" in file and not "~" in file:
                        file_data.append(file)

                # filter onzin uit de dict voor makkelijkere processing later
                if not "Schema" in root:
                    paths_dict[root] = file_data

            country_return_list.append(paths_dict)
        
    return country_return_list

In [ ]:
# functie testruimte
selected_countries = ["ParlaMint-NL.TEI"]

answer = get_xml_files(selected_countries)

print(answer)

In [ ]:
# class voor person
@dataclass
class Person:
    p_id: str
    name: str
    party: str
    sex: str

# {
#     “#personname”: “Person Name”,
#     “MORE INFO”: “LATER” //TODO
# }

# haalt nodige informatie uit het overkoepelende bestand dat per land bestaat
# NOTE: Deze functie gaat later overige nodige info over personen en partijen ook meegeven
# TODO: https://stackoverflow.com/questions/33215515/use-character-like-%C3%AB-and-%C3%A7-in-python-file
# goede characteromvorming schrijven zodat puntjes op speciale letters niet vervormen
def extract_info_xml(root, file):
    os.chdir(r"C:\Users\Asher\Documents\School\_Scriptie\Data\Data_Extract")
    
    path = os.path.join(root, file)
    person_dict, party_dict = {}, {}
    
     # lees het bestand
    with open(path, "r", encoding="utf-8") as file:
        # read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # combine the lines in the list into a string
        content = "".join(content)
        
        bs_content = bs(content, "lxml")
    
        # maakt dict met --> partij id (key) : partij naam (value)
        for party in bs_content.find_all("org"):
            
            # definieer partij afkorting en volledige naam
            part_y_1 = party.find_all("orgname")[-1].text #party.get("xml:id").split(".")[-1]
            part_y_2 = party.find_all("orgname")
            
            # kies met voorkeur de engelse variant van party
            for orgname in part_y_2:
                if orgname.get("full") == "yes" and orgname.get("xml:lang") == "en":
                    part_y_2 = orgname.text
                    break
            
            # kies anders de full variant
            else:
                
                for orgname in part_y_2:
                    
                    if orgname.get("full") == "yes":
                        part_y_2 = orgname.text
                        break
                
                # pak de eerste partijnaam als er geen betere zijn
                else:
                    part_y_2 = part_y_2[0].text
            
            # kijk welke langer is en zet die achteraan
            if len(part_y_1) > len(part_y_2):
                party_dict[party.get("xml:id")] = f"{part_y_2} ({part_y_1})" 
            else:
                party_dict[party.get("xml:id")] = f"{part_y_1} ({part_y_2})"
    
    # krijg de juiste achternaam
    def get_surname(person):
        # pak de 2e achternaam als die er is (find_all -1 kan niet omdat er in dat geval ook 4 kunnen zijn)
        if len(person.find_all('surname')) == 1:
            return person.find('surname').text
        else:
            return person.find_all('surname')[1].text
        
    # vind de juiste partij
    def get_affiliation(person):
        
        # ga van nieuw-oud door affiliations heen
        for affiliation in reversed(person.find_all("affiliation")):
    
            # return party als die ook in party_dict staat
            if affiliation.get('ref') is not None and affiliation.get('ref').strip("#") in party_dict:
                
                return party_dict[affiliation.get('ref').strip("#")]
    
    # ga door alle user info heen
    persons = bs_content.find_all("person")
    for person in persons:
        
        # NOTE: person.find_all('surname')[-1] voor laatste surname
        if person.find("affiliation") != None:
            
            pers = Person(person.get("xml:id"), 
                          (person.find('forename').text + " " + get_surname(person)),
                          get_affiliation(person),
                          person.find("sex").get('value'))
            
            person_dict[person.get("xml:id")] = pers
        
        # dit zorgt er voor dat mensen zonder partij als None worden gezet
        elif person.find('forename') != None:
            
            pers = Person(person.get("xml:id"), 
                          (person.find('forename').text + " " + get_surname(person)),
                          "None",
                          (person.find("sex").get('value') if person.find("sex") is not None else "None"))
            
            person_dict[person.get("xml:id")] = pers
                
        # neem xml:id als naam
        else:

            pers = Person(person.get("xml:id"), 
                          person.get("xml:id"),
                          "None",
                          "None")
            
            person_dict[person.get("xml:id")] = pers
            
    return person_dict

In [ ]:
# functie testruimte
# root, file = "ParlaMint-BG.TEI", "ParlaMint-BG.xml"
# root, file = "ParlaMint-PL.TEI", "ParlaMint-PL.xml"
root, file = "ParlaMint-FR.TEI", "ParlaMint-FR.xml"

test_person_dict = extract_info_xml(root, file)

print(test_person_dict)

In [ ]:
# NOTE: 9 landen zijn momenteel vertaald

# haalt CSV translation files op
def extract_translated_csv(country):
    os.chdir(r"C:\Users\Asher\Documents\School\_Scriptie\Data\Data_Translated_CSV")  

    potential_file = country + ".csv"
    
    # kijk of er een vertaald bestand voor het land beschikbaar is
    if potential_file in os.listdir():
        df = pd.read_csv(potential_file)
        
        return df.set_index("sentence_id")
    else:
        return None

In [ ]:
# functie testruimte
test_translated_csv = extract_translated_csv("BG")

display(test_translated_csv)

test_translated_csv.loc[test_translated_csv.index == "ParlaMint-BG_2014-10-27.seg2.1"]["text"][0]

In [ ]:
# haal jaar maand en dag uit filename
def extract_file_date(file_name):
    
    year_month_day = re.search(r"\d{4}-\d{2}-\d{2}", file_name)
    year, month, day = year_month_day[0].split("-")
    return year, month, day

In [ ]:
# functie testruimte
test_file = "ParlaMint-BG_2014-10-27.xml"

test_jaar, test_maand, test_dag = extract_file_date(test_file)
print(f"file: {test_file} \n - jaar: {test_jaar} \n - maand: {test_maand} \n - dag: {test_dag}")

In [ ]:
# haal de nodige data uit de xml en zet het in een list dict
def extract_debate_xml(root, file, country_info_dict, cty, translation_df=None, count=0, index="search"):
    os.chdir(r"C:\Users\Asher\Documents\School\_Scriptie\Data\Data_Extract")
    
    path = os.path.join(root, file)
    content_dict_list = []
    
    # lees het bestand
    with open(path, "r", encoding="utf-8") as file:
        # read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # combine the lines in the list into a string
        content = "".join(content)
        bs_content = bs(content, "lxml")
    
    # stel datum vast (behaLve HR, want zij besluiten om het anders te doen)
    if not cty == "HR":
        
        year, month, day = extract_file_date(str(file))
    
    # alleen in het geval van hongariij (kon het maar efficienter)
    else:
    
        year, month, day = bs_content.find("bibl").find("date").get("from").split("-")
        
    # ga door alle segmenten info heen
    userlines = bs_content.find_all("u")
    for line in userlines:
        
        content_dict, es_content_dict = {}, {}
        
        # prs[0] = naam, prs[1] = partij, prs[2] = gender
        prs = None
        
        # definieer segment, content
        seg = line.get("xml:id")
        cnt = str(line.text).strip().replace('\n', ' ')
        cnt_s = unidecode(cnt)
        
        if line.get("who") != None:
            prs = country_info_dict[line.get("who").replace('#', '')]
            
            content_dict["person"] = prs.name
            content_dict["person_simplified"] = unidecode(prs.name)
            content_dict["party"] = prs.party
            content_dict["gender"] = prs.sex
        
        content_dict["segment"] = line.get("xml:id")
        content_dict["year"] = year
        content_dict["month"] = month
        content_dict["day"] = day
        content_dict["position"] = line.get("ana")
        content_dict["country"] = cty
        content_dict["content"] = cnt
        content_dict["content_simplified"] = cnt_s.replace("\'", "")
        
        # check of er een vertaling beschikbaar is en sla die dan op
#         if isinstance(translation_df, pd.DataFrame):
#             content_dict["content_translated"] = translation_df.loc[translation_df.index == seg]["text"][0]
        
        # voeg nodige info toe aan elke xml_content_dict line voor bulk uploads
        count += 1
        
        es_content_dict["_index"] = index
        es_content_dict["_id"] = count
        es_content_dict["_source"] = content_dict
    
        # add the line to the list of lines
        content_dict_list.append(es_content_dict)
    
    return content_dict_list, count

In [ ]:
# functie testruimte
root, file = "ParlaMint-BG.TEI", "ParlaMint-BG_2014-10-27.xml"
country_info_dict = extract_info_xml("ParlaMint-BG.TEI", "ParlaMint-BG.xml")

# root, file = "ParlaMint-HR.TEI", "ParlaMint-HR_S01.xml"
# country_info_dict = extract_info_xml("ParlaMint-HR.TEI", "ParlaMint-HR.xml")


country = root.replace("ParlaMint-", "").replace(".TEI", "")
test_json = extract_debate_xml(root, file, country_info_dict, country)#, extract_translated_csv(country))

print(test_json[0])

In [ ]:
# upload xml files (progressie)

# --1.0 upload xml in stukken
# id_count = 0

# for line in test_json:
#     id_count += 1
#     resp = es.index(index="search", id=id_count, document=json.dumps(line))

# --1.1 upload xml in BULK
# helpers.bulk(es, test_json)

# --1.2
# geef een landfolder en alle info in die folder wordt verwerkt en geupload in es
def upload_xmls(selected_countries):
    
    # vraag alle xml files op van de gegeven land(en)
    countries = get_xml_files(selected_countries)
    
    # id counter voor elastic
    cnt = 0
    
    # loop door elk land heen
    for country in countries:
        
        # split hoofdfolder van mogelijke subfolders van jaren
        rootfolder = (k := next(iter(country)), country.pop(k))
        
        # verzamel person info
        country_info = extract_info_xml(rootfolder[0], rootfolder[1][0])
        
        # country label (bv: GB of NL of BE)
        cty = rootfolder[0].replace("ParlaMint-", "").replace(".TEI", "")
        
        # check of er wel jaarfolders zijn
        if len(rootfolder[1]) == 1:

            for year_folder in country:

                for file in country[year_folder]:
                    # verkrijg een verwerkt xml debat file en update counter
                    processed_xml, cnt = extract_debate_xml(year_folder, file, country_info, cty, None, cnt)

#                     print(file + " successfully parsed")
                    helpers.bulk(es, processed_xml)

        # geen jaarfolders
        else:

            # haal de personendata uit de filelijst
            rootfolder[1].pop(0)

            for file in rootfolder[1]:
                # verkrijg een verwerkt xml debat file en update counter
                processed_xml, cnt = extract_debate_xml(rootfolder[0], file, country_info, cty, None, cnt)

#                 print(file + " successfully parsed")
                helpers.bulk(es, processed_xml)

        print("---------------------------------------------------------------\n\n")
        print(cty + " is uploaded to elastic\n\n")
        print("---------------------------------------------------------------\n\n")
    
    return

In [ ]:
# upload testruimte
# TODO: als extract_translated_csv() werkt dan toevoegen in extract_debate_xml()
# extract_debate_xml(root, file, country_info_dict, country, extract_translated_csv(country))
# extract_debate_xml(root, file, country_info_dict, cty, translation_df=None, cnt=0, index="search")

# selected_countries = ['ParlaMint-BE.TEI',
#                       'ParlaMint-BG.TEI',
#                       'ParlaMint-CZ.TEI',
#                       'ParlaMint-DK.TEI',
#                       'ParlaMint-ES.TEI',
#                       'ParlaMint-FR.TEI',
#                       'ParlaMint-GB.TEI',
#                       'ParlaMint-HR.TEI',
#                       'ParlaMint-HU.TEI',
#                       'ParlaMint-IS.TEI',
#                       'ParlaMint-IT.TEI',
#                       'ParlaMint-LT.TEI',
#                       'ParlaMint-NL.TEI',
#                       'ParlaMint-LV.TEI',
#                       'ParlaMint-PL.TEI',
#                       'ParlaMint-SI.TEI',
#                       'ParlaMint-TR.TEI']

# selected_countries = ["ParlaMint-NL.TEI",
#                       'ParlaMint-FR.TEI',
#                       'ParlaMint-GB.TEI',
#                       'ParlaMint-BE.TEI']

# upload_xmls(selected_countries)

In [ ]:
# elasticsearch settings and mapping to index
# es.indices.close(index="search")

# es.indices.put_settings(index="search", body={
#     "analysis": {
#         "analyzer": {
#             "default": {
#                 "type": "custom",
#                 "tokenizer": "standard",
#                 "filter": [
#                     "lowercase",
#                     "asciifolding"
#                 ]
#             }
#         }
#     }
# })

# es.indices.put_mapping(index="search", body={
#     "properties" :{
#         "cnt" : {
#             "type" : "text",
#             "analyzer" : "default"
#         }
#     }
# })

# es.indices.open(index="search")

In [ ]:
# NOTE: input is {search_key: value, search_key2: value2}
# query function
def query(search_dict):
    processed_search_list = []
    
    # loop door alle search elements heen
    for k, v in search_dict.items():
        processed_search_list.append({"match_phrase" : {k : v}})
        
    # stel de uitkomst samen
    result = es.search(
    index="search", 
    body={
#         "size" : 3, # <------ HARDCODED VOOR TESTING --------
        "query": {
            "bool" : {
                "must": processed_search_list,},},})
    
    return result

In [ ]:
# query testruimte
start_time = time.time()

# de query zelf
# test_result = query({"content_simplified":"ete", 
#                      "person_simplified":"FreDeriC BARBIER", 
#                      "country":"FR"})

test_result = query({"content_simplified":"groen",
                     "party":"SP",
                     "year":"2018"})

print("Got %d Hits:" % test_result['hits']['total']['value'])
for hit in test_result['hits']['hits']:
    print("%(person)s (%(party)s) \n(%(year)s-%(month)s-%(day)s) %(segment)s:\n %(content)s \n" % hit["_source"])

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# verwijder inhoud "search" index
# es.options(ignore_status=[400,404]).indices.delete(index='search')